In [1]:
import re

generator_files = [
"PW/src/allocate_fft_large.f90.generator",
"PW/src/allocate_locpot_large.f90.generator",
"PW/src/force_lc_large.f90.generator",
"PW/src/gradcorr_large.f90.generator",
"PW/src/init_vloc_large.f90.generator",
"PW/src/v_of_rho_large.f90.generator",
#"Modules/xc_rVV10_large.f90.generator",
#"Modules/xc_vdW_DF_large.f90.generator",
"Modules/recvec_subs_large.f90.generator",
"Modules/cell_base_large.f90.generator",
]
eqe_new_root = "/home/alessandro/QE/hack2018//eqe_new/"
qe_current_root = "/home/alessandro/QE/hack2018//qe_current/"

In [2]:
original_function_name = [
"v_xc_meta",
"v_xc",
"v_h",
"v_hubbard",
"v_hubbard_nc",
"v_h_of_rho_r",
#"hessian",
#"grad_dot",
#"gradient",
"gradcorr",
#"gradrho",
"init_vloc",
"allocate_locpot",
#"vdW_DF (module)",
#"xc_vdW_DF",
#"stress_vdW_DF",
#"initialize_spline_interpolation",
#"interpolate_kernel",
#"numerical_gradient",
"allocate_fft",
"force_lc",
#"xc_rVV10",
#"stress_rVV10",
#"stress_rVV10_gradient",
#"initialize_spline_interpolation",
#"interpolate_kernel",
#"numerical_gradient",
"gshells"
]

patched_function_name = [
"v_xc_meta_large",
"v_xc_large",
"v_h_large",
"v_hubbard_large",
"v_hubbard_nc_large",
"v_h_of_rho_r_large",
#"hessian_large",
#"grad_dot_large",
#"gradient_large",
"gradcorr_large",
#"gradrho_large",
"init_vloc_large",
"allocate_locpot_large",
#"vdW_DF_large",
#"xc_vdW_DF_large",
#"stress_vdW_DF_large",
#"initialize_spline_interpolation_large",
#"interpolate_kernel_large",
#"numerical_gradient_large",
"allocate_fft_large",
"force_lc_large_large",
#"xc_rVV10_large",
#"stress_rVV10_large",
#"stress_rVV10_gradient_large",
#"initialize_spline_interpolation_large",
#"interpolate_kernel_large",
#"numerical_gradient_large",
"gshells_large"
]

In [3]:
def get_block_logic(block_name, block_type, filename):
    
    module_rgx = re.compile(r'^\s*module\s+(?!procedure\b)(\w+)',re.IGNORECASE)
    endmodule_rgx = re.compile(r'^\s*end\s*module(?:\s+(\w+))?',re.IGNORECASE)
    subroutine_rgx = re.compile(r'^\s*subroutine\s+(\w+)\s*(?:\((.*)\))?',re.IGNORECASE)
    endsubroutine_rgx = re.compile(r'^\s*end\s*subroutine(?:\s+(\w+))?',re.IGNORECASE)
    function_rgx = re.compile(r'^\s*(?:(?:real|complex|integer|character|logical).*\s+)?function\s+(\w+)\s*(?:\((.*?)\))?\s*(?:result\s*(?:\((.*?)\)))?'                          ,re.IGNORECASE)
    endfunction_rgx = re.compile(r'^\s*end\s*function(?:\s+(\w+))?',re.IGNORECASE)
    implicitnone_rgx = re.compile(r'^\s*implicit\s+none\s*$',re.IGNORECASE)
    
    if block_type == "subroutine":
        start_rgx = subroutine_rgx
        end_rgx = endsubroutine_rgx
    elif block_type == "function":
        start_rgx = function_rgx
        end_rgx = endfunction_rgx
    elif block_type == "module":
        start_rgx = module_rgx
        end_rgx = endmodule_rgx
    else:
        raise Exception("Unknown block_type {} in block {} in file {}".format(block_type, block_name, filename))
    
    logic_lines = []
    with open(qe_current_root+filename) as f:
        
        in_block = False
        in_logic = False
        
        for line in f:
            
            # check if entering a subroutine
            # or a function
            mo0 = re.match(start_rgx, line)
            if mo0 is not None and in_block:
                raise Exception("Error fetching the logic of block {} in {}".format(block_name, filename))
            if mo0 is not None:    
                if mo0.group(1).lower() == block_name.lower():
                    #print("routine start")
                    in_block = True

            if in_block:
                if not in_logic:
                    #print(line)
                    mo0 = re.match(implicitnone_rgx, line)
                    if mo0 is not None:
                        #print("logic start implicit")
                        in_logic = True
                    else:
                        loline = line.lower()
                        #for typ in ["integer, complex, character, real, logical, type"]:
                        if loline.strip().startswith(("integer", "complex", "character", "real", "logical", "type")):
                            #
                            #print("logic start declaration")
                            in_logic = True
                            logic_lines.append(line)
                else:
                    mo0 = re.match(end_rgx, line)
                    if mo0 is not None:
                        #print("procedure end")
                        return logic_lines
                    else:
                        logic_lines.append(patch_line(line))
                
    raise Exception("Error fetching the logic of module {} in {}".format(block_name, filename))

In [4]:
def _get_routine_logic(routine_name, filename):
    subroutine_rgx = re.compile(r'^\s*subroutine\s+(\w+)\s*(?:\((.*)\))?',re.IGNORECASE)
    endsubroutine_rgx = re.compile(r'^\s*end\s*subroutine(?:\s+(\w+))?',re.IGNORECASE)
    implicitnone_rgx = re.compile(r'^\s*implicit\s+none\s*$',re.IGNORECASE)
    function_rgx = re.compile(r'^\s*(?:(?:real|complex|integer|character|logical).*\s+)?function\s+(\w+)\s*(?:\((.*?)\))?\s*(?:result\s*(?:\((.*?)\)))?'                          ,re.IGNORECASE)
    endfunction_rgx = re.compile(r'^\s*end\s*function(?:\s+(\w+))?',re.IGNORECASE)
    logic_lines = []
    with open(qe_current_root+filename) as f:
        
        in_routine = False
        in_logic = False
        
        for line in f:
            
            # check if entering a subroutine
            # or a function
            mo0 = re.match(subroutine_rgx, line)
            mo1 = re.match(function_rgx, line)
            if (mo0 is not None or mo1 is not None) and in_routine:
                raise Exception("Error fetching the logic of routine {} in {}".format(routine_name, filename))
            if mo0 is not None:    
                if mo0.group(1).lower() == routine_name.lower():
                    #print("routine start")
                    in_routine = True
            if mo1 is not None:
                if mo1.group(1).lower() == routine_name.lower():
                    #print("function start")
                    in_routine = True
            

            if in_routine:
                if not in_logic:
                    #print(line)
                    mo0 = re.match(implicitnone_rgx, line)
                    if mo0 is not None:
                        #print("logic start implicit")
                        in_logic = True
                    else:
                        loline = line.lower()
                        #for typ in ["integer, complex, character, real, logical, type"]:
                        if loline.strip().startswith(("integer", "complex", "character", "real", "logical", "type")):
                            #
                            #print("logic start declaration")
                            in_logic = True
                            logic_lines.append(line)
                else:
                    mo0 = re.match(endsubroutine_rgx, line)
                    mo1 = re.match(endfunction_rgx, line)
                    if mo0 is not None or mo1 is not None:
                        #print("procedure end")
                        return logic_lines
                    else:
                        logic_lines.append(patch_line(line))
                
    raise Exception("Error fetching the logic of routine {} in {}".format(routine_name, filename))

In [5]:
import json
def generator_to_f90(filename):
    f90file = filename[:-10]
    prev_line = ""
    with open(eqe_new_root+filename) as fin, open(eqe_new_root+f90file,"w") as fout:
        for line in fin:
            if "!hack2018" in line.lower():
                line = line.replace("!hack2018", "")
                #spline = line.split()
                list_line = json.loads(line)
                if len(list_line) == 3:
                    block_name, original_file, block_type = list_line
                    logic_lines = get_block_logic(block_name, block_type, original_file)
                else:
                    raise Exception("!hack2018 note is wrong: ", list_line, filename)
                for line1 in logic_lines:
                    fout.write(line1)
            else:
                pass
                fout.write(line)
            prev_line = line
    print(f90file)
#get_routine_logic("v_xc", "./v_of_rho.f90")

In [6]:
#original_function_name = ["v_xc_meta", "v_xc", "v_h"]
#patched_function_name = ["v_xc_meta_large", "v_xc_large", "v_h_large"]
def patch_line(line):
    # if there is a Dense invfft or fft, convert it to Custom
    # also, if the non-large version of a routine that has been patched to large
    # is called, replace the call with the large version
    loline = line.lower()
    if "fwfft" in loline or "invfft" in loline:
        return line.replace("Dense", "Custom")
    else:
        for i, sub_name in enumerate(original_function_name):
            subcall_rgx = re.compile(r'^\s*call\s+{}\s*(?:\((.*)\))?\s*$'.format(sub_name),re.IGNORECASE)
            mo = re.match(subcall_rgx, line)
            if mo is not None:
                return loline.replace(sub_name, patched_function_name[i])
    return line
    

In [8]:
for generator in generator_files:
    #print(generator)
    generator_to_f90(generator)

PW/src/allocate_fft_large.f90
PW/src/allocate_locpot_large.f90
PW/src/force_lc_large.f90
PW/src/gradcorr_large.f90
PW/src/init_vloc_large.f90
PW/src/v_of_rho_large.f90
Modules/recvec_subs_large.f90
Modules/cell_base_large.f90
